# OCT image classification - Xception

Experiment with the number of trainable layers. Use 10% of train data

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
# for filename in filenames:
#       print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
image_path = '/kaggle/input/kermany2018/oct2017/OCT2017 '
oct_csv_path = '/kaggle/input/oct-csv/'
train_dir = image_path + "/train/"
valid_dir = image_path + "/val/"
test_dir = image_path + "/test/"

In [3]:
classes = ['CNV', 'DME', 'DRUSEN', 'NORMAL']
cols = [x.upper() for x in classes]
dirs = [train_dir, valid_dir, test_dir]
label = {0: 'CNV', 1: 'DME', 2: 'DRUSEN', 3: 'NORMAL'}
IMG_SIZE = 224

# if we should read the directory structre, if False then use the CSV files already saved
# Once you generate the csv files you should probably download them and re-upload into kaggle and set this to FALSE
REGEN = False

In [4]:
def create_df (path, classes=classes):
  df = pd.DataFrame(columns=['FILENAME', 'CNV', 'DME', 'DRUSEN', 'NORMAL'])
  for sub_dir in classes:
    condition = {'NORMAL': 0, 'CNV': 0, 'DME':0, 'DRUSEN': 0}
    files = os.listdir(path + sub_dir)
    if (sub_dir== 'NORMAL'):
      condition['NORMAL'] = 1
    elif (sub_dir == 'CNV'):
      condition['CNV'] = 1
    elif (sub_dir == 'DME'):
      condition['DME'] = 1
    else:
      condition['DRUSEN']= 1
    for f in files:
      df = df.append({'FILENAME': path +  sub_dir  + "/" + f, 
                      'NORMAL': condition['NORMAL'], 
                      'CNV': condition['CNV'],
                      'DME': condition['DME'],
                      'DRUSEN': condition['DRUSEN']}, ignore_index=True)
  return df

In [5]:
# Generting the DataFrames of the filenames
# this is primarily used so we can sub-sample files easier for the different training strategies
if (REGEN):
  train_df = create_df(train_dir)
  valid_df = create_df(valid_dir)
  test_df = create_df(test_dir)
  train_df.to_csv("train_data.csv")
  valid_df.to_csv("valid_data.csv")
  test_df.to_csv("test_data.csv")
else:
  train_df = pd.read_csv(oct_csv_path + "train_data.csv")
  valid_df = pd.read_csv(oct_csv_path + "valid_data.csv")
  test_df = pd.read_csv(oct_csv_path + "test_data.csv")

In [6]:
print ("Training Data: ", train_df.shape)
print ("Validation Data: ", valid_df.shape)
print ("Test Data: ", test_df.shape)

Training Data:  (83484, 5)
Validation Data:  (36, 5)
Test Data:  (972, 5)


In [7]:
# Printing out the # of samples for each subsample percentage 
print ("Trainig Data percentages:")
print (" 1% ==> ", int(.01 * train_df.shape[0]))
print (" 5% ==> ", int(.05 * train_df.shape[0]))
print ("10% ==> ", int(.1  * train_df.shape[0]))
print ("25% ==> ", int(.25 * train_df.shape[0]))
print ("50% ==> ", int(.5  * train_df.shape[0]))
print ("75% ==> ", int(.75 * train_df.shape[0]))
print ("90% ==> ", int(.9  * train_df.shape[0]))
print ("98% ==> ", int(.98 * train_df.shape[0]))

Trainig Data percentages:
 1% ==>  834
 5% ==>  4174
10% ==>  8348
25% ==>  20871
50% ==>  41742
75% ==>  62613
90% ==>  75135
98% ==>  81814


In [8]:
# Sampling 10% of the data
sample = train_df.sample(frac=0.1, random_state=10, axis=0)
sample.shape

(8348, 5)

In [9]:
# determine class weights to feed into neural network during training
def get_classweight(df):
  total = df.shape[0]
  num_norm = df['NORMAL'].sum()
  num_cnv = df['CNV'].sum()
  num_dme = df['DME'].sum()
  num_drusen = df['DRUSEN'].sum()
  norm_weight = (1/num_norm) * (total/4)
  cnv_weight = (1/num_cnv) * (total/4)
  dme_weight = (1/num_dme) * (total/4)
  drusen_weight = (1/num_drusen) * (total/4)
  class_weight = {0 : cnv_weight, 1: dme_weight,
                  2 : drusen_weight, 3: norm_weight}
  return class_weight

In [10]:
class_weight = get_classweight(sample)
class_weight

{0: 0.5654294229206177,
 1: 1.8387665198237886,
 2: 2.371590909090909,
 3: 0.7899318697956094}

In [11]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import tensorflow.keras.applications as app
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
train_image_datagen = ImageDataGenerator(rotation_range=90, width_shift_range=[-.1,.1], height_shift_range=[-.1,.1],
                                         shear_range=0.25, zoom_range=0.3, horizontal_flip=True,
                                         vertical_flip=True, rescale = 1./255., validation_split=0.1)

# Setting the imgages to come from the dataframe where we specify the filenames and columns to use for "labels"
train_imgs = train_image_datagen.flow_from_dataframe(sample, directory=None, x_col='FILENAME', y_col=cols, subset="training",
                                        class_mode="raw", target_size=(IMG_SIZE,IMG_SIZE), batch_size=32, seed=10)
valid_imgs = train_image_datagen.flow_from_dataframe(sample, directory=None, x_col='FILENAME', y_col=cols, subset="validation",
                                        class_mode="raw", target_size=(IMG_SIZE,IMG_SIZE), batch_size=32, seed=10)

Found 7514 validated image filenames.
Found 834 validated image filenames.


In [13]:
# Creating the model based on Xception Network
input_layer = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = app.xception.Xception(include_top=False, weights="imagenet", input_shape=(IMG_SIZE,IMG_SIZE,3))


x = base_model(input_layer)
x = keras.layers.GlobalAveragePooling2D()(x)
output = keras.layers.Dense(4, activation="softmax")(x)


# set base_model layers trainable

base_model.trainable = True

set_trainable = False

# set layers to trainable
for layer in base_model.layers:
    if layer in base_model.layers[116:] :
        set_trainable = True
        layer.trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False 


model = keras.Model(inputs=input_layer, outputs=output)


83689472/83683744 [==============================] - 4s 0us/step


In [14]:
# layer trainable status of base_model layers
layers = [(layer, layer.name, layer.trainable) for layer in base_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])  

,Layer Type,Layer Name,Layer Trainable
0,<tensorflow.python.keras.engine.input_layer.In...,input_2,False
1,<tensorflow.python.keras.layers.convolutional....,block1_conv1,False
2,<tensorflow.python.keras.layers.normalization_...,block1_conv1_bn,False
3,<tensorflow.python.keras.layers.core.Activatio...,block1_conv1_act,False
4,<tensorflow.python.keras.layers.convolutional....,block1_conv2,False
...,...,...,...
127,<tensorflow.python.keras.layers.normalization_...,block14_sepconv1_bn,True
128,<tensorflow.python.keras.layers.core.Activatio...,block14_sepconv1_act,True
129,<tensorflow.python.keras.layers.convolutional....,block14_sepconv2,True
130,<tensorflow.python.keras.layers.normalization_...,block14_sepconv2_bn,True


In [15]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
xception (Functional)        (None, 7, 7, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 8196      
Total params: 20,869,676
Trainable params: 6,796,580
Non-trainable params: 14,073,096
_________________________________________________________________


In [16]:
# This code did not work, it caused I/O Error 5:
# model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics='accuracy')
model.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Creating a checkpoint to save the best model so that we can reload it once training is complete
checkpoint_cb = keras.callbacks.ModelCheckpoint("oct_xception10per16Layers.h5", save_best_only=True)
# Adding an an early stop callback to avoid overfitting in case the model is not improving after 5 consescutive epochs
earlystop_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [18]:
%%timeit
history = model.fit(train_imgs,  steps_per_epoch=1000, epochs=100, verbose=1, validation_data=valid_imgs, 
                    class_weight=class_weight, callbacks=[checkpoint_cb, earlystop_cb])

Epoch 1/100
1000/1000 [==============================] - 201s 190ms/step - loss: 0.7542 - accuracy: 0.7289 - val_loss: 0.5093 - val_accuracy: 0.8237
Epoch 1/100
1000/1000 [==============================] - 128s 128ms/step - loss: 0.5363 - accuracy: 0.8175 - val_loss: 0.4301 - val_accuracy: 0.8333
Epoch 1/100
1000/1000 [==============================] - 130s 130ms/step - loss: 0.4917 - accuracy: 0.8267 - val_loss: 0.5976 - val_accuracy: 0.7494
Epoch 1/100
1000/1000 [==============================] - 130s 130ms/step - loss: 0.4334 - accuracy: 0.8484 - val_loss: 0.4182 - val_accuracy: 0.8537
Epoch 1/100
1000/1000 [==============================] - 128s 128ms/step - loss: 0.4164 - accuracy: 0.8569 - val_loss: 0.3428 - val_accuracy: 0.8861
Epoch 1/100
1000/1000 [==============================] - 140s 140ms/step - loss: 0.4089 - accuracy: 0.8579 - val_loss: 0.3934 - val_accuracy: 0.8537
Epoch 1/100
1000/1000 [==============================] - 134s 134ms/step - loss: 0.3805 - accuracy: 0.8647

In [19]:
test_image_datagen = ImageDataGenerator(rescale = 1./255.)

test_imgs = test_image_datagen.flow_from_dataframe(test_df, directory=None, x_col='FILENAME', y_col=cols, validate_filenames=True,
                                        class_mode="raw", target_size=(224,224), batch_size=32)

Found 968 validated image filenames.


/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 4 invalid image filename(s) in x_col="FILENAME". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [20]:
model.load_weights("oct_xception10per16Layers.h5")
model.evaluate(test_imgs)

31/31 [==============================] - 10s 323ms/step - loss: 0.1438 - accuracy: 0.9442


[0.14375635981559753, 0.9442148804664612]

In [21]:
results = model.predict(test_imgs)
results

array([[8.8283348e-01, 4.8784600e-03, 1.1062596e-01, 1.6620808e-03],
       [9.7375676e-05, 1.1792942e-02, 1.9855233e-02, 9.6825445e-01],
       [8.9110667e-01, 3.5926172e-05, 1.0885371e-01, 3.7574380e-06],
       ...,
       [2.0393253e-04, 9.5454007e-01, 3.1983379e-02, 1.3272631e-02],
       [9.6804374e-01, 1.6639766e-03, 3.0282319e-02, 9.9932095e-06],
       [9.9890888e-01, 2.0440039e-04, 8.8002492e-04, 6.6842408e-06]],
      dtype=float32)

In [22]:
from sklearn.metrics import classification_report, plot_confusion_matrix, confusion_matrix

In [23]:
model.save("oct_xception10per16Layers.h5")